This notebook explains how to use the <i>PETSubjectEvaluation</i> class to evaluate Gloabl and Local metrics of chosen subject data. The example computes all metrics between <i>original (full-dose)</i> and <i>noisy (DRF=10)</i> data, and calculates a final score indicating the performance by calculating a weighted sum of all global and local metric results.

The final score is a weighed sum of both global and local metric results, and the local MAPE value is again a weighed sum of percentage errors of all local metric results. The score is calculated based on the following formula:
* SCORE = 0.5 * (0.4 * PSNR + 0.2 * SSIM - 0.4 * NRMSE) - 0.5 * (local_MAPE)


All metrics involved are as follows:

### Global metrics
* NRMSE 
* PSNR
* SSIM

### Local metrics
* SUV_mean
* SUV_max
* Local PSNR
* Total lesion metabolism
* FirstOrder: RootMeanSquared
* FirstOrder: 90Percentile
* FirstOrder: Median
* GLRLM: HighGrayLevelRunEmphasis
* GLSZM: ZonePercentage
* GLCM: JointAverage

## Evaluation

Once all nifti files are prepared, you can refer to the following to do the evaluation. Here we provide an example evaluating the original, full-dose PET and the low-dose PET with DRF=10

In [1]:
from ..common.evaluation import PETSubjectEvaluation

In [3]:
# Init evaluation object
evaluator = PETSubjectEvaluation(
    # Directory of validation full-dose nifti files
    real_dir='',
    # Directory of nifti results generated of your model, here uses noisy DRF=10 dataset
    gen_dir='', 
    # Directory of nifti organ masks
    # Organs of interest are liver, heart, right kidney, left kidney, labelled 01, 02, 03, 04 respectively
    mask_dir='',
    # CSV file containing test subjects' metadata used for computing SUV metrics, etc
    meta_file='data/Test/meta.csv',
    # Number of evaluating subjects, here uses 2 subject's data
    num_subject=2
)

With the instance now being initilized, follow the instructions below to compute:
* Global metrics of all subjects
* Local metrics of all subjects
* All together, global and local metrics of all subjects

In [4]:
# Compute global metrics of all predicted (model generated) subjects
global_df = evaluator.compute_global_metrics()
global_df

,PID,NRMSE,PSNR,SSIM
0,Anonymous_ANO_20220726_1946371_114021,0.001500,56.480365,0.999331
0,Anonymous_ANO_20220726_2120094_095417,0.000418,67.570818,0.999926


In [5]:
# Compute local metrics 
# Percentage error between predicted results and mask, across all organs)
# Set to verbose, containing all local features
local_df = evaluator.compute_local_metrics(verbose=True)
local_df

,Organ,PID,firstorder_RootMeanSquared_percentage_error,firstorder_RootMeanSquared_pred,firstorder_RootMeanSquared_real,firstorder_90Percentile_percentage_error,firstorder_90Percentile_pred,firstorder_90Percentile_real,firstorder_Median_percentage_error,firstorder_Median_pred,...,SUV_mean_real,SUV_max_percentage_error,SUV_max_pred,SUV_max_real,PSNR_percentage_error,PSNR_pred,PSNR_real,TotalLesionMetabolism_percentage_error,TotalLesionMetabolism_pred,TotalLesionMetabolism_real
0,liver,Anonymous_ANO_20220726_1946371_114021,1.824040,1.089067,1.069558,16.136357,1.352548,1.164621,2.479337,1.041100,...,1.066802,63.233286,2.111316,1.293435,0.269226,4442.356184,4430.428314,0.580883,13.620558,13.700140
0,heart,Anonymous_ANO_20220726_1946371_114021,2.561822,1.216015,1.247986,4.629634,1.726638,1.810456,2.840909,1.056099,...,1.166279,0.033206,2.655688,2.656570,2.897412,2005.633947,2065.479402,0.581843,13.617703,13.697400
0,kidneyRight,Anonymous_ANO_20220726_1946371_114021,1.513017,1.811456,1.784457,0.330040,2.574871,2.566400,1.002517,1.394016,...,1.637391,21.747106,5.853102,4.807590,0.144124,1388.141509,1390.145036,0.580837,13.617344,13.696901
0,kidneyLeft,Anonymous_ANO_20220726_1946371_114021,3.821851,1.343014,1.396381,0.303951,1.736344,1.741637,5.362115,1.199030,...,1.346730,5.709952,3.691495,3.492098,5.423331,2255.708975,2385.058608,0.583082,13.618275,13.698146
0,liver,Anonymous_ANO_20220726_2120094_095417,1.939758,1.295504,1.270852,17.829258,1.652892,1.402785,2.236812,1.240329,...,1.265973,54.455137,2.459188,1.592170,0.226088,7961.387852,7979.428393,0.683904,13.036066,13.125834
0,heart,Anonymous_ANO_20220726_2120094_095417,4.596985,0.997366,0.953532,19.672137,1.253542,1.047480,0.459427,0.952956,...,0.950550,52.477258,1.891048,1.240216,2.521460,6142.386585,5991.317901,0.680891,13.032424,13.121769
0,kidneyRight,Anonymous_ANO_20220726_2120094_095417,9.698054,9.876469,9.003322,6.061187,15.133008,14.268187,1.200452,1.917473,...,5.179442,16.421686,49.302677,42.348362,3.664695,4161.170828,4014.067495,0.681117,13.033991,13.123376
0,kidneyLeft,Anonymous_ANO_20220726_2120094_095417,13.165551,4.586912,4.053276,14.860001,5.780826,5.032932,2.561948,1.745710,...,2.726145,27.664402,36.970402,28.959053,5.321375,5647.676829,5362.327272,0.679166,13.035774,13.124914


In [6]:
# Set to organ-verbose, for each subject contain avg. values across all organs 
local_df = evaluator.compute_local_metrics(organ_verbose=True)
local_df

,firstorder_RootMeanSquared_percentage_error,firstorder_90Percentile_percentage_error,firstorder_Median_percentage_error,glrlm_HighGrayLevelRunEmphasis_percentage_error,glszm_ZonePercentage_percentage_error,glcm_JointAverage_percentage_error,SUV_mean_percentage_error,SUV_max_percentage_error,PSNR_percentage_error,TotalLesionMetabolism_percentage_error
PID,,,,,,,,,,
Anonymous_ANO_20220726_1946371_114021,2.430183,5.349996,2.921219,25.186670,9.902505,17.434291,2.183523,22.680888,2.183523,0.581661
Anonymous_ANO_20220726_2120094_095417,7.350087,14.605646,1.614660,27.609437,6.084997,16.606517,2.933405,37.754621,2.933405,0.681270


In [7]:
# Set to non-verbose (default), return MAPE value only
# The is the avg. (weighed) sum of all local metric percentage errors
local_df = evaluator.compute_local_metrics()
local_df

,MAPE
PID,
Anonymous_ANO_20220726_1946371_114021,0.854890
Anonymous_ANO_20220726_2120094_095417,1.237337


In [8]:
# Compute global and local metrics at once
# And calculate score = 0.5 * (0.4 * PSNR + 0.2 * SSIM - 0.4 * NRMSE) - 0.5 * MAPE
# Save results to csv file
all_df = evaluator.evaluate_all(save_to_file=True)
all_df

Evaluating subject Anonymous_ANO_20220726_1946371_114021.nii.gz
Evaluating subject Anonymous_ANO_20220726_2120094_095417.nii.gz


,PID,NRMSE,PSNR,SSIM,MAPE,SCORE
0,Anonymous_ANO_20220726_1946371_114021,0.001500,56.480365,0.999331,0.854890,10.968261
0,Anonymous_ANO_20220726_2120094_095417,0.000418,67.570818,0.999926,1.237337,12.995404
